# SHARK QUEST
## New Kids On The Block



# TODO

## Selecting
- Select relevant columns
- Analyse relevant columns

## Cleaning :
- Cast to appropriate data types
    - General cleaning
        - identify duplicates
            - fuzzy
        - removing duplicates
            - remove
            - merge
        - handling null values
            - remove
            - replace
        - manipulating strings
        - formatting the data.

- Wrong inputs

## Hypothesis
- Location
- Time
- Species

## Business cases

In [2]:
import pandas as pd
import re

# Load the data
raw_data = pd.read_excel('GSAF5.xls')

#new_data.to_csv("shark_data.csv")

In [3]:
raw_data.shape

(6969, 23)

In [4]:
raw_data.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#PRINT ALL COLUMNS WITH FIRST VALUE
raw_data.iloc[0]

Date                                           15 Mar 2024
Year                                                2024.0
Type                                            Unprovoked
Country                                          AUSTRALIA
State                                           Queensland
Location                                     Bargara Beach
Activity                                          Swimming
Name                                        Brooklyn Sauer
Sex                                                      F
Age                                                     13
Injury            Minor injuries to back, abdomen and legs
Unnamed: 11                                            NaN
Time                                                 16h00
Species                                        Tiger shark
Source                               Yahoo News, 3/15/2024
pdf                                                    NaN
href formula                                           N

In [16]:
# Source, PDF, Case Number as possible duplicate finder or year / date fill
relevant_columns = ['Date','Year','Type','Country','State','Location','Activity','Name','Sex','Age','Injury','Time','Species','Source', 'pdf','href formula','href' ]

In [ ]:
#PRINT UNIQUE
sample = raw_data['Unnamed: 22'].unique()

for i in range(1000) :
    print(sample[i])

In [ ]:
#PRINT MULTIPLE COLUMNS

data = raw_data[['Case Number','Case Number.1']]
print(data)
for i in range(1000):
    print(data.iloc[i])


In [ ]:
# PRINT ALL MISFORMATED DATES (YEAR END \d{4})
for index, value in raw_data['Date'].items():
    #print(index,value)
    is_match = re.match('.*\d{4}$',str(value))

    if not is_match:
        print(index, value) 